# MLP Reference Implementation in Pytorch.

This implementation is based on Pytorch's FashionMNIST example in its
[quickstart](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html).

See the mlax implementation in `mlp.ipynb` notebook.

In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
import torchvision

### Load the MNIST dataset.

In [2]:
mnist_train = torchvision.datasets.MNIST(
    root="../data",
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor()
)
mnist_test = torchvision.datasets.MNIST(
    root="../data",
    train=False,
    download=True,
    transform=torchvision.transforms.ToTensor()
)
print(mnist_train.data.shape)
print(mnist_test.data.shape)

torch.Size([60000, 28, 28])
torch.Size([10000, 28, 28])


### Batch the MNIST data with Pytorch dataloaders.

In [3]:
batch_size = 128
train_dataloader = DataLoader(
    mnist_train, batch_size, shuffle=True, num_workers=6, pin_memory=True
)
test_dataloader = DataLoader(
    mnist_test, batch_size, num_workers=6, pin_memory=True
)
print(len(train_dataloader), len(test_dataloader))

469 79


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


### Define MLP using modules.
We jit-compiled the model for better performance.

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, batch):
        flattened = self.flatten(batch)
        logits = self.linear_stack(flattened)
        return logits

model = MLP().to(device)
print(model)
model = torch.jit.script(model)

MLP(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### Define loss function and optimizer.

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

### Define training and testing steps.

In [8]:
def train_step(X, y):
    loss = loss_fn(model(X), y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss

In [9]:
def test_step(X, y):
    with torch.no_grad():
        preds = model(X)
        loss = loss_fn(preds, y)
    accurate = (preds.argmax(1) == y).type(torch.int).sum()
    return loss, accurate

### Define training and testing loops.

In [10]:
def train(dataloader):
    model.train()
    train_loss = 0.0
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        train_loss += train_step(X, y).item()

    print(f"Train loss: {train_loss / len(dataloader)}")

In [11]:
def test(dataloader):
    model.eval()
    test_loss, accurate = 0.0, 0
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        loss, acc = test_step(X, y)
        test_loss += loss.item()
        accurate += acc.item()
    
    print(f"Test loss: {test_loss / len(dataloader)}, accuracy: {accurate / len(dataloader.dataset)}")

In [12]:
def train_loop(
    train_dataloader,
    test_dataloader,
    epochs,
    test_every
):
    for i in range(epochs):
        epoch = (i + 1)
        print(f"Epoch {epoch}\n----------------")
        train(train_dataloader)
        if (epoch % test_every == 0):
            test(test_dataloader)
        print(f"----------------")

### Train MLP on MNIST dataset.

In [13]:
train_loop(train_dataloader, test_dataloader, 30, 5)

Epoch 1
----------------
Train loss: 0.7000591684696771
----------------
Epoch 2
----------------
Train loss: 0.2604155678516512
----------------
Epoch 3
----------------
Train loss: 0.19238585626075008
----------------
Epoch 4
----------------
Train loss: 0.1489394486093445
----------------
Epoch 5
----------------
Train loss: 0.11897123144713165
Test loss: 0.11014873180921556, accuracy: 0.968
----------------
Epoch 6
----------------
Train loss: 0.09886586911547413
----------------
Epoch 7
----------------
Train loss: 0.08339812173676897
----------------
Epoch 8
----------------
Train loss: 0.0700083428552982
----------------
Epoch 9
----------------
Train loss: 0.06094559369835137
----------------
Epoch 10
----------------
Train loss: 0.05288976564733332
Test loss: 0.07200376054823186, accuracy: 0.9765
----------------
Epoch 11
----------------
Train loss: 0.04601647154187788
----------------
Epoch 12
----------------
Train loss: 0.040308763445821655
----------------
Epoch 13
------